In [1]:
# ! .venv\Scripts\pip install darts
# ! .venv\Scripts\pip install dask dask[distributed]
# ! .venv\Scripts\pip install bokeh dask[dataframe] pyarrow

In [44]:
from dask.distributed import Client

# Initialize the client
client = Client(n_workers=4, memory="2GB")  # This starts the Dask scheduler and worker processes
client

2024-10-21 22:15:54,163 - distributed.nanny - ERROR - Failed to start process
Traceback (most recent call last):
  File "c:\Users\a4293604\Documents\deep-learning\.venv\Lib\site-packages\distributed\nanny.py", line 452, in instantiate
    result = await self.process.start()
             ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\a4293604\Documents\deep-learning\.venv\Lib\site-packages\distributed\nanny.py", line 758, in start
    msg = await self._wait_until_connected(uid)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\a4293604\Documents\deep-learning\.venv\Lib\site-packages\distributed\nanny.py", line 904, in _wait_until_connected
    raise msg["exception"]
  File "c:\Users\a4293604\Documents\deep-learning\.venv\Lib\site-packages\distributed\nanny.py", line 955, in run
    worker = worker_factory()
    ^^^^^^^^^^^^^^^^^
  File "c:\Users\a4293604\Documents\deep-learning\.venv\Lib\site-packages\distributed\worker.py", line 715, in __init__
    ServerNode.__init__(
   

RuntimeError: Nanny failed to start.

In [38]:
import dask.dataframe as dd
df = dd.read_parquet('TRAIN_Reco_2021_2022_2023.parquet.gzip').reset_index()

In [39]:
df['ExecutionTime'] = dd.to_datetime(df['ExecutionTime'])
numerical_columns = ['high', 'low', 'close', 'volume']
df[numerical_columns] = df[numerical_columns].astype('float16')
df.dtypes

ExecutionTime    datetime64[ns, Europe/Berlin]
ID                             string[pyarrow]
high                                   float16
low                                    float16
close                                  float16
volume                                 float16
dtype: object

In [40]:
train_start_date = '2022-01-01'
train_end_date = '2023-06-30'

val_start_date = '2023-07-01'
val_end_date = '2023-12-31'  # Adjust if you have data beyond 2023

# Step 4: Split the data into training and validation sets
train_df = df[(df['ExecutionTime'] >= train_start_date) & (df['ExecutionTime'] <= train_end_date)]
val_df = df[(df['ExecutionTime'] >= val_start_date) & (df['ExecutionTime'] <= val_end_date)]

In [41]:
print(train_df["ID"].nunique(), val_df["ID"].nunique())

<dask_expr.expr.Scalar: expr=(DropDuplicates(frame=(Filter(frame=Assign(frame=Assign(frame=ResetIndex(frame=ReadParquetFSSpec(60b046e)))), predicate=Assign(frame=Assign(frame=ResetIndex(frame=ReadParquetFSSpec(60b046e))))['ExecutionTime'] >= 2022-01-01 & Assign(frame=Assign(frame=ResetIndex(frame=ReadParquetFSSpec(60b046e))))['ExecutionTime'] <= 2023-06-30))['ID'], split_every=False)).count(), dtype=int64> <dask_expr.expr.Scalar: expr=(DropDuplicates(frame=(Filter(frame=Assign(frame=Assign(frame=ResetIndex(frame=ReadParquetFSSpec(60b046e)))), predicate=Assign(frame=Assign(frame=ResetIndex(frame=ReadParquetFSSpec(60b046e))))['ExecutionTime'] >= 2023-07-01 & Assign(frame=Assign(frame=ResetIndex(frame=ReadParquetFSSpec(60b046e))))['ExecutionTime'] <= 2023-12-31))['ID'], split_every=False)).count(), dtype=int64>


In [43]:
train_df.compute()

ArrowMemoryError: malloc of size 1073741824 failed

In [6]:
train_df.set_index('ExecutionTime', inplace=True)
val_df.set_index('ExecutionTime', inplace=True)

def create_lag_rolling_features(df):
    # Lag features
    for column in ['low', 'high', 'close', 'volume']:
        for lag in range(1, 11):  # Create 10 lags
            df[f'{column}_lag_{lag}'] = df[column].shift(lag)
    
    # Rolling window features (rolling mean of the last 10 periods)
    for column in ['low', 'high', 'close', 'volume']:
        df[f'{column}_rolling_mean_10'] = df[column].rolling(window=10).mean()
    
    return df

In [7]:
# Apply the lag and rolling window function to each asset group separately in the training set
train_df = train_df.groupby('ID').apply(create_lag_rolling_features)

# Apply the lag and rolling window function to each asset group separately in the validation set
val_df = val_df.groupby('ID').apply(create_lag_rolling_features)

C:\Users\a4293604\AppData\Local\Temp\ipykernel_14520\2639633781.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_df = train_df.groupby('ID').apply(create_lag_rolling_features)
C:\Users\a4293604\AppData\Local\Temp\ipykernel_14520\2639633781.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  val_df = val_df.groupby('ID').apply(create_lag_rolling_features)


In [8]:
# Handle missing values resulting from lagging
train_df.dropna(inplace=True)
val_df.dropna(inplace=True)

In [9]:
train_df.head()

ID  high  low  close  volume  \
ID      ExecutionTime                                                  
Fri00Q1 2022-01-01 02:30:00+01:00  Fri00Q1   0.0  0.0    0.0     0.0   
        2022-01-01 02:45:00+01:00  Fri00Q1   0.0  0.0    0.0     0.0   
        2022-01-01 03:00:00+01:00  Fri00Q1   0.0  0.0    0.0     0.0   
        2022-01-01 03:15:00+01:00  Fri00Q1   0.0  0.0    0.0     0.0   
        2022-01-01 03:30:00+01:00  Fri00Q1   0.0  0.0    0.0     0.0   

                                   low_lag_1  low_lag_2  low_lag_3  low_lag_4  \
ID      ExecutionTime                                                           
Fri00Q1 2022-01-01 02:30:00+01:00        0.0        0.0        0.0        0.0   
        2022-01-01 02:45:00+01:00        0.0        0.0        0.0        0.0   
        2022-01-01 03:00:00+01:00        0.0        0.0        0.0        0.0   
        2022-01-01 03:15:00+01:00        0.0        0.0        0.0        0.0   
        2022-01-01 03:30:00+01:00        0.0        0.0        0.0        0.0   

                                   low_lag_5  ...  volume_lag_5  volume_lag_6  \
ID      ExecutionTime                         ...                               
Fri00Q1 2022-01-01 02:30:00+01:00        0.0  ...           0.0           0.0   
        2022-01-01 02:45:00+01:00        0.0  ...           0.0           0.0   
        2022-01-01 03:00:00+01:00        0.0  ...           0.0           0.0   
        2022-01-01 03:15:00+01:00        0.0  ...           0.0           0.0   
        2022-01-01 03:30:00+01:00        0.0  ...           0.0           0.0   

                                   volume_lag_7  volume_lag_8  volume_lag_9  \
ID      ExecutionTime                                                         
Fri00Q1 2022-01-01 02:30:00+01:00           0.0           0.0           0.0   
        2022-01-01 02:45:00+01:00           0.0           0.0           0.0   
        2022-01-01 03:00:00+01:00           0.0           0.0           0.0   
        2022-01-01 03:15:00+01:00           0.0           0.0           0.0   
        2022-01-01 03:30:00+01:00           0.0           0.0           0.0   

                                   volume_lag_10  low_rolling_mean_10  \
ID      ExecutionTime                                                   
Fri00Q1 2022-01-01 02:30:00+01:00            0.0                  0.0   
        2022-01-01 02:45:00+01:00            0.0                  0.0   
        2022-01-01 03:00:00+01:00            0.0                  0.0   
        2022-01-01 03:15:00+01:00            0.0                  0.0   
        2022-01-01 03:30:00+01:00            0.0                  0.0   

                                   high_rolling_mean_10  \
ID      ExecutionTime                                     
Fri00Q1 2022-01-01 02:30:00+01:00                   0.0   
        2022-01-01 02:45:00+01:00                   0.0   
        2022-01-01 03:00:00+01:00                   0.0   
        2022-01-01 03:15:00+01:00                   0.0   
        2022-01-01 03:30:00+01:00                   0.0   

                                   close_rolling_mean_10  \
ID      ExecutionTime                                      
Fri00Q1 2022-01-01 02:30:00+01:00                    0.0   
        2022-01-01 02:45:00+01:00                    0.0   
        2022-01-01 03:00:00+01:00                    0.0   
        2022-01-01 03:15:00+01:00                    0.0   
        2022-01-01 03:30:00+01:00                    0.0   

                                   volume_rolling_mean_10  
ID      ExecutionTime                                      
Fri00Q1 2022-01-01 02:30:00+01:00                     0.0  
        2022-01-01 02:45:00+01:00                     0.0  
        2022-01-01 03:00:00+01:00                     0.0  
        2022-01-01 03:15:00+01:00                     0.0  
        2022-01-01 03:30:00+01:00                     0.0  

[5 rows x 49 columns]

In [16]:
train_df.shape

(35150304, 49)

In [15]:
val_df.shape

(11411904, 49)

In [10]:
from sklearn.preprocessing import MinMaxScaler

# List of columns to scale
columns_to_scale = ['high', 'low', 'close', 'volume']

# Create copies of the DataFrames
train_df_scaled = train_df.copy()
val_df_scaled = val_df.copy()

# Dictionary to store scalers for each asset
scalers = {}

# Assets present in training data
assets_in_train = train_df_scaled.index.get_level_values('ID').unique()

for asset in assets_in_train:
    # Training data for this asset
    asset_train_data = train_df_scaled.loc[asset, columns_to_scale]
    
    # Initialize and fit the scaler
    scaler = MinMaxScaler()
    scaled_train_values = scaler.fit_transform(asset_train_data)
    
    # Replace training data with scaled values
    train_df_scaled.loc[asset, columns_to_scale] = scaled_train_values
    
    # Store the scaler
    scalers[asset] = scaler
    
    # Check if the asset exists in validation data
    if asset in val_df_scaled.index.get_level_values('ID'):
        asset_val_data = val_df_scaled.loc[asset, columns_to_scale]
        
        # Transform validation data
        scaled_val_values = scaler.transform(asset_val_data)
        
        # Replace validation data with scaled values
        val_df_scaled.loc[asset, columns_to_scale] = scaled_val_values
    else:
        # Asset not in validation data; no action needed
        pass

# Handle assets present only in validation data
assets_in_val = val_df_scaled.index.get_level_values('ID').unique()
assets_only_in_val = set(assets_in_val) - set(assets_in_train)

for asset in assets_only_in_val:
    print(f"Warning: Asset {asset} is present in validation data but not in training data. Skipping scaling for this asset.")
    # Decide how to handle these assets
    # For example, you could drop them:
    val_df_scaled = val_df_scaled.drop(asset, level='ID')

# # Reset index if necessary
# train_df_scaled = train_df_scaled.reset_index()
# val_df_scaled = val_df_scaled.reset_index()

# Now proceed with your modeling using train_df_scaled and val_df_scaled


In [12]:
train_df_scaled.shape

(35150304, 49)

In [16]:
train_df_scaled["ID"].nunique()

672

In [17]:
new_train_scaled_df = train_df_scaled
new_val_scaled_df = val_df_scaled

In [18]:
new_train_scaled_df['ID_numeric'] = new_train_scaled_df['ID'].astype('category').cat.codes
new_train_scaled_df.head()

ID  high      low  close  volume  \
ID      ExecutionTime                                                      
Fri00Q1 2022-01-01 02:30:00+01:00  Fri00Q1   0.0  0.04895    0.0     0.0   
        2022-01-01 02:45:00+01:00  Fri00Q1   0.0  0.04895    0.0     0.0   
        2022-01-01 03:00:00+01:00  Fri00Q1   0.0  0.04895    0.0     0.0   
        2022-01-01 03:15:00+01:00  Fri00Q1   0.0  0.04895    0.0     0.0   
        2022-01-01 03:30:00+01:00  Fri00Q1   0.0  0.04895    0.0     0.0   

                                   low_lag_1  low_lag_2  low_lag_3  low_lag_4  \
ID      ExecutionTime                                                           
Fri00Q1 2022-01-01 02:30:00+01:00        0.0        0.0        0.0        0.0   
        2022-01-01 02:45:00+01:00        0.0        0.0        0.0        0.0   
        2022-01-01 03:00:00+01:00        0.0        0.0        0.0        0.0   
        2022-01-01 03:15:00+01:00        0.0        0.0        0.0        0.0   
        2022-01-01 03:30:00+01:00        0.0        0.0        0.0        0.0   

                                   low_lag_5  ...  volume_lag_6  volume_lag_7  \
ID      ExecutionTime                         ...                               
Fri00Q1 2022-01-01 02:30:00+01:00        0.0  ...           0.0           0.0   
        2022-01-01 02:45:00+01:00        0.0  ...           0.0           0.0   
        2022-01-01 03:00:00+01:00        0.0  ...           0.0           0.0   
        2022-01-01 03:15:00+01:00        0.0  ...           0.0           0.0   
        2022-01-01 03:30:00+01:00        0.0  ...           0.0           0.0   

                                   volume_lag_8  volume_lag_9  volume_lag_10  \
ID      ExecutionTime                                                          
Fri00Q1 2022-01-01 02:30:00+01:00           0.0           0.0            0.0   
        2022-01-01 02:45:00+01:00           0.0           0.0            0.0   
        2022-01-01 03:00:00+01:00           0.0           0.0            0.0   
        2022-01-01 03:15:00+01:00           0.0           0.0            0.0   
        2022-01-01 03:30:00+01:00           0.0           0.0            0.0   

                                   low_rolling_mean_10  high_rolling_mean_10  \
ID      ExecutionTime                                                          
Fri00Q1 2022-01-01 02:30:00+01:00                  0.0                   0.0   
        2022-01-01 02:45:00+01:00                  0.0                   0.0   
        2022-01-01 03:00:00+01:00                  0.0                   0.0   
        2022-01-01 03:15:00+01:00                  0.0                   0.0   
        2022-01-01 03:30:00+01:00                  0.0                   0.0   

                                   close_rolling_mean_10  \
ID      ExecutionTime                                      
Fri00Q1 2022-01-01 02:30:00+01:00                    0.0   
        2022-01-01 02:45:00+01:00                    0.0   
        2022-01-01 03:00:00+01:00                    0.0   
        2022-01-01 03:15:00+01:00                    0.0   
        2022-01-01 03:30:00+01:00                    0.0   

                                   volume_rolling_mean_10  ID_numeric  
ID      ExecutionTime                                                  
Fri00Q1 2022-01-01 02:30:00+01:00                     0.0           0  
        2022-01-01 02:45:00+01:00                     0.0           0  
        2022-01-01 03:00:00+01:00                     0.0           0  
        2022-01-01 03:15:00+01:00                     0.0           0  
        2022-01-01 03:30:00+01:00                     0.0           0  

[5 rows x 50 columns]

In [21]:
new_train_scaled_df.columns

Index(['ID', 'high', 'low', 'close', 'volume', 'low_lag_1', 'low_lag_2',
       'low_lag_3', 'low_lag_4', 'low_lag_5', 'low_lag_6', 'low_lag_7',
       'low_lag_8', 'low_lag_9', 'low_lag_10', 'high_lag_1', 'high_lag_2',
       'high_lag_3', 'high_lag_4', 'high_lag_5', 'high_lag_6', 'high_lag_7',
       'high_lag_8', 'high_lag_9', 'high_lag_10', 'close_lag_1', 'close_lag_2',
       'close_lag_3', 'close_lag_4', 'close_lag_5', 'close_lag_6',
       'close_lag_7', 'close_lag_8', 'close_lag_9', 'close_lag_10',
       'volume_lag_1', 'volume_lag_2', 'volume_lag_3', 'volume_lag_4',
       'volume_lag_5', 'volume_lag_6', 'volume_lag_7', 'volume_lag_8',
       'volume_lag_9', 'volume_lag_10', 'low_rolling_mean_10',
       'high_rolling_mean_10', 'close_rolling_mean_10',
       'volume_rolling_mean_10', 'ID_numeric'],
      dtype='object')

In [22]:
import pandas as pd
from darts import TimeSeries
from darts.models import RNNModel

In [27]:
# Remove timezone information from the 'ExecutionTime' column
# new_train_scaled_df['ExecutionTime'] = pd.to_datetime(new_train_scaled_df['ExecutionTime']).dt.tz_localize(None)
# new_train_scaled_df.index = new_train_scaled_df.index.tz_localize(None)

# Remove 'ID' from the index, keep 'ExecutionTime' as the index
new_train_scaled_df = new_train_scaled_df.reset_index(level='ID', drop=True)


In [32]:
new_train_scaled_df = new_train_scaled_df.reset_index()
new_train_scaled_df['ExecutionTime'] = pd.to_datetime(new_train_scaled_df['ExecutionTime']).dt.tz_localize(None)

new_train_scaled_df.head()

,index,ExecutionTime,ID,high,low,close,volume,low_lag_1,low_lag_2,low_lag_3,...,volume_lag_6,volume_lag_7,volume_lag_8,volume_lag_9,volume_lag_10,low_rolling_mean_10,high_rolling_mean_10,close_rolling_mean_10,volume_rolling_mean_10,ID_numeric
0,0,2022-01-01 02:30:00,Fri00Q1,0.0,0.04895,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1,2022-01-01 02:45:00,Fri00Q1,0.0,0.04895,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,2,2022-01-01 03:00:00,Fri00Q1,0.0,0.04895,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,3,2022-01-01 03:15:00,Fri00Q1,0.0,0.04895,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,4,2022-01-01 03:30:00,Fri00Q1,0.0,0.04895,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [37]:
new_train_scaled_df = new_train_scaled_df.drop(columns=['index'])

MemoryError: Unable to allocate 2.88 GiB for an array with shape (44, 35150304) and data type float16

In [35]:
import gc

# Delete unused variables
# del large_unused_dataframe

# Explicitly call garbage collection
gc.collect()


2278

In [36]:
target_columns = ['high', 'low', 'close', 'volume']
feature_columns = [col for col in new_train_scaled_df.columns if col not in target_columns + ['ID_numeric', 'ExecutionTime']]

# Step 2: Convert the dataframe into Darts TimeSeries objects
# TimeSeries for target variables (multivariate time series)
targets = TimeSeries.from_dataframe(new_train_scaled_df, time_col='ExecutionTime', value_cols=target_columns)

# TimeSeries for covariates (including lags, rolling means, and ID_numeric)
covariates = TimeSeries.from_dataframe(new_train_scaled_df, time_col='ExecutionTime', value_cols=feature_columns + ['ID_numeric'])


MemoryError: Unable to allocate 268. MiB for an array with shape (35150304,) and data type datetime64[ns]

c:\Users\a4293604\Documents\deep-learning\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Features
X_train = new_train_scaled_df.drop(['ID', 'ExecutionTime', 'high', 'low', 'close', 'volume'], axis=1)

# Targets
y_train = new_train_scaled_df[['high', 'low', 'close', 'volume']]


In [ ]:
model = RNNModel(
    input_chunk_length=15,  # Number of past time steps to use as input
    output_chunk_length=10,  # Number of future time steps to predict
    model="LSTM",  # You can choose between "RNN", "LSTM", and "GRU"
    n_epochs=100,
    batch_size=32,
    random_state=42
)